In [1]:
import os
import time

import bluesky.plans as bp
import dxchange
import numpy as np
import tomopy
from bluesky.run_engine import RunEngine
from xpdan.vend.callbacks import LiveTable
from ophyd.sim import SynSignal, hw, SynSignalWithRegistry
from xpdan.vend.callbacks.zmq import Publisher
from xpdconf.conf import glbl_dict

# create fake hardware (motors and detector)
hw = hw()
fname = os.path.expanduser("tooth.h5")

proj, flat, dark, theta = dxchange.read_aps_32id(fname, sino=(0, 1))

proj = tomopy.normalize(proj, flat, dark)

rot_center = tomopy.find_center(proj, theta, init=290, ind=0, tol=0.5)
proj2 = proj
theta_motor = hw.motor1
theta_motor.kind = "hinted"


class FullField:
    def __call__(self, *args, **kwargs):
        v = theta_motor.get()[0]
        out = proj2[int(v), :, :]
        time.sleep(.5)
        return out

f = FullField()
det = SynSignalWithRegistry(f, name="img", labels={"detectors"},
                            save_path='/home/christopher/dev/provenance-driven-ldrd/demo/raw_data')
det.kind = "hinted"

# create run engine, link with ZMQ system
RE = RunEngine()
p = Publisher(glbl_dict["inbound_proxy_address"], prefix=b"raw")
t = RE.subscribe(p)

# build tomo scan locations
def build_scan():
    l = [0., 90.]
    for i in range(8):
        ll = l.copy()
        interval = sorted(set(ll))[1] / 2
        for lll in ll:
            j = lll + interval
            j = round(j, 0)
            if j not in l and j <= 180:
                l.append(j)
    return l

/home/christopher/mc/envs/dp_dev/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [2]:
# maybe load some of these before the demo? (Finished turkey in the oven and all)

In [3]:
shots=16

In [5]:
RE(
    bp.list_scan(
        [det],
        theta_motor,
        build_scan()[:shots],
        md={
            "tomo": {
                "type": "full_field",
                "rotation": "motor1",
                "center": rot_center,
            },
            'bt_piLast': 'Wright',
            'analysis_stage': 'raw',
            'sample_type': 'tooth',
            'sample_name': 'dinosaur tooth'
        },
    ), LiveTable(['motor1'])
)

/home/christopher/dev/xpdAn/xpdan/vend/callbacks/core.py:318: UserWarning: The key img will be skipped because LiveTable does not know how to display the dtype array
  "".format(k, dk_entry["dtype"])


+-----------+------------+------------+
|   seq_num |       time |     motor1 |
+-----------+------------+------------+
|         1 | 11:28:18.2 |      0.000 |
|         2 | 11:28:18.7 |     90.000 |
|         3 | 11:28:19.3 |     45.000 |
|         4 | 11:28:19.8 |    135.000 |
|         5 | 11:28:20.3 |     22.000 |
|         6 | 11:28:20.8 |    112.000 |
|         7 | 11:28:21.3 |     68.000 |
|         8 | 11:28:21.9 |    158.000 |
|         9 | 11:28:22.4 |     11.000 |
|        10 | 11:28:22.9 |    101.000 |
|        11 | 11:28:23.4 |     56.000 |
|        12 | 11:28:23.9 |    146.000 |
|        13 | 11:28:24.5 |     33.000 |
|        14 | 11:28:25.0 |    123.000 |
|        15 | 11:28:25.5 |     79.000 |
|        16 | 11:28:26.1 |    169.000 |
+-----------+------------+------------+
generator list_scan ['0fd54125'] (scan num: 2)


('0fd54125-590f-4e06-9eab-9a60cb77fd9c',)